<a href="https://colab.research.google.com/github/Evan-Magoo/CmpSc-472-Project1/blob/main/472_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Parallel Sorting

## Multi-Threading

In [178]:
%%writefile parallel_sort_multithreading.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <pthread.h>
#include <time.h>
#include <sys/resource.h>

// Array Size
#define ARRAY_SIZE 131072

// Global Variables
int array[ARRAY_SIZE];
int chunk_size;
int NUM_THREADS;

// Returns current memory usage by program
long get_memory_usage() {
    FILE* fp = fopen("/proc/self/status", "r");
    if (fp == NULL) {
        perror("Error opening /proc/self/status");
        return 0;
    }
    char line[256];
    long vm_rss = -1;

    // Scan file until VmRSS field is found
    while (fgets(line, sizeof(line), fp) != NULL) {
        if (strncmp(line, "VmRSS:", 6) == 0) {
            sscanf(line + 6, "%ld", &vm_rss);
            break;
        }
    }

    fclose(fp);
    return vm_rss;
}

// Recursieve QuickSort Implementation
void quickSort(int *array, int low, int high) {
    if (low < high) {
        int pivot = array[low];
        int i = low;
        int j = high;

        // Partitioning
        while (i < j) {
            // Find first element larger than pivot
            while (array[i] <= pivot && i <= high - 1) {
                i++;
            }
            // Find first element smaller than pivot
            while (array[j] > pivot && j >= low + 1) {
                j--;
            }
            if (i < j) {
                // Swap elements
                int temp = array[i];
                array[i] = array[j];
                array[j] = temp;
            }
        }

        // Place pivot in correct position
        int temp = array[low];
        array[low] = array[j];
        array[j] = temp;

        // Recursive calls
        quickSort(array, low, j - 1);
        quickSort(array, j + 1, high);
    }
}

// Thread routine for assigning local chunks and sorting them
void* chunk_sorting(void* arg) {
    int thread_id = *(int *)arg;
    int start = thread_id * chunk_size;
    int end = 0;

    // Last thread gets last elements
    if (thread_id == NUM_THREADS - 1) {
        end = ARRAY_SIZE - 1;
    } else {
        end = start + chunk_size - 1;
    }

    printf("\tThread %d: Sorting %d to %d\n", thread_id, start, end);
    fflush(stdout);

    // Copy local chunk into temporary array
    int local_size = end - start + 1;
    int *local_array = malloc(local_size * sizeof(int));
    memcpy(local_array, &array[start], local_size * sizeof(int));

    // Sort local chunk
    quickSort(local_array, 0, local_size - 1);

    // Copy sorted chunk back into main array
    memcpy(&array[start], local_array, local_size * sizeof(int));

    free(local_array);
    pthread_exit(NULL);
}

// Merge two sorted subarrays into single sorted array
void merge(int *array, int low, int mid, int high) {
    int n1 = mid - low + 1;
    int n2 = high - mid;

    // Create temp arrays
    int *left = malloc(n1 * sizeof(int));
    int *right = malloc(n2 * sizeof(int));

    // Copy data to temp arrays
    for (int i = 0; i < n1; i++) {
        left[i] = array[low + i];
    }
    for (int j = 0; j < n2; j++) {
        right[j] = array[mid + 1 + j];
    }

    // Merge until one array runs out
    int i = 0;
    int j = 0;
    int k = low;
    while (i < n1 && j < n2) {
        if (left[i] <= right[j]) {
            array[k++] = left[i++];
        } else {
            array[k++] = right[j++];
        }
    }

    // Copy remaining elements
    while (i < n1) {
        array[k++] = left[i++];
    }
    while (j < n2) {
        array[k++] = right[j++];
    }

    free(left);
    free(right);
}

// Main Method
int main(void) {
    printf("------------------------------------------------------------------------------------------------------------------------\n");

    int thread_count[] = {1, 2, 4, 8};  // Thread counts
    double performance[4];              // For storing execution times
    long memory_usage[4];               // For storing memory usage

    struct timespec c_start, c_end;
    long mem_before, mem_after;

    // Loop through the different thread counts
    for (int t = 0; t <4; t++) {
        NUM_THREADS = thread_count[t];
        chunk_size = ARRAY_SIZE / NUM_THREADS;

        // Label thread configurations
        if (NUM_THREADS == 1){
            printf(" - %d THREAD:\n", NUM_THREADS);
        } else {
            printf(" - %d THREADS:\n", NUM_THREADS);
        }

        // Generate & Fill Array
        srand(42);
        for (int i = 0; i < ARRAY_SIZE; i++) {
            array[i] = rand() % 100;
        }

        // Print sample of unsorted array
        printf("    - Before sorting (first 20 elements):\n\t");
        for (int i = 0; i < 20; i++) {
            printf("%d ", array[i]);
        }
        printf("\n\n");

        // Record memory and time before sorting
        mem_before = get_memory_usage();
        clock_gettime(CLOCK_MONOTONIC, &c_start);

        // ---- Map Phase ------------------------------------------------------
        // Each thread sorts one chunk of array
        pthread_t threads[NUM_THREADS];
        int thread_ids[NUM_THREADS];

        printf("    - Sorting:\n");
        for (int i = 0; i < NUM_THREADS; i++) {
            thread_ids[i] = i;
            pthread_create(&threads[i], NULL, chunk_sorting, &thread_ids[i]);
        }

        // Wait for all threads to complete
        for (int i = 0; i < NUM_THREADS; i++) {
            pthread_join(threads[i], NULL);
        }
        printf("\n\t - All threads finished -\n");


        // ---- Reduce Phase ---------------------------------------------------
        // Merge sorted chunks iteratively into single sorted array
        int step = chunk_size;
        while (step < ARRAY_SIZE) {
            for (int i = 0; i < ARRAY_SIZE; i += 2 * step) {
                int low = i;
                int mid = i + step - 1;
                int high = 0;
                if ((i + 2 * step - 1) < ARRAY_SIZE) {
                    high = i + 2 * step - 1;
                } else {
                    high = ARRAY_SIZE - 1;
                }
                merge(array, low, mid, high);
            }
            step *= 2; // Merge larger sections each pass
        }

        // Record memory and time after sorting
        mem_after = get_memory_usage();
        clock_gettime(CLOCK_MONOTONIC, &c_end);

        // Print sample after sorting
        printf("\n    - After sorting (first 20 elements):\n\t");
        for (int i = 0; i < 20; i++) {
            printf("%d ", array[i]);
        }
        printf("\n\n");

        // Calculate Execution time
        double execution_time = (c_end.tv_sec - c_start.tv_sec) + (c_end.tv_nsec - c_start.tv_nsec) / 1e9;
        printf("    - Execution Time: %f sec", execution_time);
        performance[t] = execution_time;

        // Calculate memory usage
        long mem_usage = mem_after - mem_before;
        printf("\n\n    - Memory Before: %ld KB", mem_before);
        printf("\n    - Memory After: %ld KB", mem_after);
        printf("\n\n    - Memory Delta: %ld KB", mem_usage);
        memory_usage[t] = mem_usage;

        printf("\n------------------------------------------------------------------------------------------------------------------------\n");
    }

    // Display performance summary for all thread configs
    printf("\nPerformance Summary:\n");
    printf("Threads\t   Time (s)\tMem Delta (KB)\n");
    for (int t = 0; t < 4; t++) {
        printf("%d\t   %.6f\t%ld\n", thread_count[t], performance[t], memory_usage[t]);
    }

    return 0;
}


Overwriting parallel_sort_multithreading.c


### Results

In [179]:
!gcc parallel_sort_multithreading.c -o parallel_sort_multithreading -lpthread
!./parallel_sort_multithreading

------------------------------------------------------------------------------------------------------------------------
 - 1 THREAD:
    - Before sorting (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Sorting:
	Thread 0: Sorting 0 to 131071

	 - All threads finished -

    - After sorting (first 20 elements):
	0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

    - Execution Time: 0.293143 sec

    - Memory Before: 1996 KB
    - Memory After: 2564 KB

    - Memory Delta: 568 KB
------------------------------------------------------------------------------------------------------------------------
 - 2 THREADS:
    - Before sorting (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Sorting:
	Thread 0: Sorting 0 to 65535
	Thread 1: Sorting 65536 to 131071

	 - All threads finished -

    - After sorting (first 20 elements):
	0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

    - Execution Time: 0.126942 sec

    - Memory B

## Multi-Procressing

In [189]:
%%writefile parallel_sort_multitprocessing.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <unistd.h>
#include <sys/mman.h>
#include <time.h>

// Array Size
#define ARRAY_SIZE 131072

// Global Variables
int *array;
int chunk_size;
int NUM_PROCESSES;
long *shared_mem_usage;

// Returns current memory usage by program
long get_memory_usage() {
    FILE* fp = fopen("/proc/self/status", "r");
    if (fp == NULL) {
        perror("Error opening /proc/self/status");
        return 0;
    }
    char line[256];
    long vm_rss = -1;

    // Scan file until VmRSS field is found
    while (fgets(line, sizeof(line), fp) != NULL) {
        if (strncmp(line, "VmRSS:", 6) == 0) {
            sscanf(line + 6, "%ld", &vm_rss);
            break;
        }
    }

    fclose(fp);
    return vm_rss;
}

// Recursieve QuickSort Implementation
void quickSort(int *array, int low, int high) {
    if (low < high) {
        int pivot = array[low];
        int i = low;
        int j = high;

        // Partitioning
        while (i < j) {
            // Find first element larger than pivot
            while (array[i] <= pivot && i <= high - 1) {
                i++;
            }
            // Find first element smaller than pivot
            while (array[j] > pivot && j >= low + 1) {
                j--;
            }
            if (i < j) {
                // Swap elements
                int temp = array[i];
                array[i] = array[j];
                array[j] = temp;
            }
        }

        // Place pivot in correct position
        int temp = array[low];
        array[low] = array[j];
        array[j] = temp;

        // Recursive calls
        quickSort(array, low, j - 1);
        quickSort(array, j + 1, high);
    }
}

// Merge two sorted subarrays into single sorted array
void merge(int *array, int low, int mid, int high) {
    int n1 = mid - low + 1;
    int n2 = high - mid;

    // Create temp arrays
    int *left = malloc(n1 * sizeof(int));
    int *right = malloc(n2 * sizeof(int));

    // Copy data to temp arrays
    for (int i = 0; i < n1; i++) {
        left[i] = array[low + i];
    }
    for (int j = 0; j < n2; j++) {
        right[j] = array[mid + 1 + j];
    }

    // Merge until one array runs out
    int i = 0;
    int j = 0;
    int k = low;
    while (i < n1 && j < n2) {
        if (left[i] <= right[j]) {
            array[k++] = left[i++];
        } else {
            array[k++] = right[j++];
        }
    }

    // Copy remaining elements
    while (i < n1) {
        array[k++] = left[i++];
    }
    while (j < n2) {
        array[k++] = right[j++];
    }

    free(left);
    free(right);
}

// Main Method
int main(void) {
    printf("------------------------------------------------------------------------------------------------------------------------\n");
    int process_count[] = {1, 2, 4, 8}; // Worker configs
    double performance[4];              // For storing execution times
    long memory_usage[4];               // For storing memory usage

    struct timespec c_start, c_end;

    // Loop through the different process counts
    for (int p = 0; p < 4; p++) {
        NUM_PROCESSES = process_count[p];
        chunk_size = ARRAY_SIZE / NUM_PROCESSES;

        // Label process configurations
        if (NUM_PROCESSES == 1){
            printf(" - %d PROCESS:\n", NUM_PROCESSES);
        } else {
            printf(" - %d PROCESSES:\n", NUM_PROCESSES);
        }

        // Set up shared memory
        array = mmap(NULL, ARRAY_SIZE * sizeof(int), PROT_READ | PROT_WRITE, MAP_ANONYMOUS | MAP_SHARED, -1, 0);
        shared_mem_usage = mmap(NULL, 8 * sizeof(long), PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);

        // Generate & Fill Array
        srand(42);
        for (int i = 0; i < ARRAY_SIZE; i++) {
            array[i] = rand() % 100;
        }

        // Print sample of unsorted array
        printf("    - Before sorting (first 20 elements):\n\t");
        for (int i = 0; i < 20; i++) {
            printf("%d ", array[i]);
        }
        printf("\n\n");

        // Record time before sorting
        clock_gettime(CLOCK_MONOTONIC, &c_start);

        // ---- Map Phase ------------------------------------------------------
        // Each procress sorts one chunk of array
        printf("    - Sorting:\n");
        pid_t pids[NUM_PROCESSES];
        for (int i = 0; i < NUM_PROCESSES; i++) {
            pids[i] = fork();
            if (pids[i] == 0) {
                int start = i * chunk_size;
                int end = 0;
                if (i == NUM_PROCESSES - 1){
                    end = ARRAY_SIZE - 1;
                } else {
                    end = start + chunk_size - 1;
                }

                printf("\tProcess %d (PID=%d): sorting %d to %d\n", i, getpid(), start, end);
                fflush(stdout);

                int local_size = end - start + 1;
                int *local_array = malloc(local_size * sizeof(int));
                memcpy(local_array, &array[start], local_size * sizeof(int));
                quickSort(local_array, 0, local_size - 1);
                memcpy(&array[start], local_array, local_size * sizeof(int));\
                free(local_array);

                shared_mem_usage[i] = get_memory_usage();
                _exit(0);

            }
        }

        // Wait for all processes to complete
        for (int i = 0; i < NUM_PROCESSES; i++) {
            waitpid(pids[i], NULL, 0);
        }
        printf("\n\t - All processess finished -\n");

        // Get total memory usage of processes
        long total_memory = 0;
        for (int i = 0; i < NUM_PROCESSES; i++) {
            total_memory += shared_mem_usage[i];
        }
        printf("\n    - Total memory used by children: %ld KB\n", total_memory);
        memory_usage[p] = total_memory;

        // ---- Reduce Phase ---------------------------------------------------
        // Merge sorted chunks iteratively into a single sorted array
        int step = chunk_size;
        while (step < ARRAY_SIZE) {
            for (int i = 0; i < ARRAY_SIZE; i += 2 * step) {
                int low = i;
                int mid = i + step - 1;
                int high = (i + 2 * step - 1 < ARRAY_SIZE) ? (i + 2 * step - 1) : (ARRAY_SIZE - 1);
                merge(array, low, mid, high);
            }
            step *= 2; // Merge larger sections each pass
        }

        // Record time after sorting
        clock_gettime(CLOCK_MONOTONIC, &c_end);

        // Print sample after sorting
        printf("\n    - After sorting (first 20 elements):\n\t");
        for (int i = 0; i < 20; i++) {
            printf("%d ", array[i]);
        }
        printf("\n\n");

        // Calculate execution time
        double execution_time = (c_end.tv_sec - c_start.tv_sec) + (c_end.tv_nsec - c_start.tv_nsec) / 1e9;
        printf("    - Execution Time: %.6f sec", execution_time);
        performance[p] = execution_time;

        printf("\n------------------------------------------------------------------------------------------------------------------------\n");
    }

    printf("\nPerformance Summary:\n");
    printf("Processes  Time (s)\tMem Usage (KB)\n");
    for (int p = 0; p < 4; p++) {
        printf("%d\t   %.6f\t%ld\n", process_count[p], performance[p], memory_usage[p]);
    }

    return 0;
}

Overwriting parallel_sort_multitprocessing.c


### Results

In [190]:
!gcc parallel_sort_multitprocessing.c -o parallel_sort_multitprocessing -lpthread
!./parallel_sort_multitprocessing

------------------------------------------------------------------------------------------------------------------------
 - 1 PROCESS:
    - Before sorting (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Sorting:
	Process 0 (PID=146261): sorting 0 to 131071

	 - All processess finished -

    - Total memory used by children: 1592 KB

    - After sorting (first 20 elements):
	0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

    - Execution Time: 0.255534 sec
------------------------------------------------------------------------------------------------------------------------
 - 2 PROCESSES:
    - Before sorting (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Sorting:
	Process 0 (PID=146262): sorting 0 to 65535
	Process 1 (PID=146263): sorting 65536 to 131071

	 - All processess finished -

    - Total memory used by children: 2648 KB

    - After sorting (first 20 elements):
	0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

# Max-Value Aggregration

## Multi-Threading

In [191]:
%%writefile max_value_multithreading.c
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <time.h>
#include <string.h>

// Array Size
#define ARRAY_SIZE 131072

// Global Variables
int array[ARRAY_SIZE];
int chunk_size;
pthread_mutex_t mutex = PTHREAD_MUTEX_INITIALIZER;
int global_max;
int NUM_THREADS;

// Returns current memory usage by program
long get_memory_usage() {
    FILE* fp = fopen("/proc/self/status", "r");
    if (fp == NULL) {
        perror("Error opening /proc/self/status");
        return 0;
    }
    char line[256];
    long vm_rss = -1;

    // Scan file until VmRSS field is found
    while (fgets(line, sizeof(line), fp) != NULL) {
        if (strncmp(line, "VmRSS:", 6) == 0) {
            sscanf(line + 6, "%ld", &vm_rss);
            break;
        }
    }

    fclose(fp);
    return vm_rss;
}

// Computes maximum value within assigned chunk
void* find_local_max(void* arg) {
    int id = *(int *)arg;
    free(arg);

    // Determine start and end for thread chunk
    int start = id * chunk_size;
    int end = 0;
    if (id == NUM_THREADS - 1) {
        end = ARRAY_SIZE - 1;
    } else {
        end = start + chunk_size - 1;
    }

    printf("\tThread %d: Finding local max in %d to %d\n", id, start, end);
    fflush(stdout);

    // Compute local max
    int local_max = array[start];
    for (int i = start + 1; i <= end; i++) {
        if (array[i] > local_max) {
            local_max = array[i];
        }
    }

    // ---- Reduce Phase -------------------------------------------------------
    // Updates global max protected with mutex
    pthread_mutex_lock(&mutex);
    if (local_max > global_max) {
        global_max = local_max;
    }
    pthread_mutex_unlock(&mutex);

    pthread_exit(NULL);
}

// Main Method
int main(void) {
    printf("------------------------------------------------------------------------------------------------------------------------\n");
    int thread_count[] = {1, 2, 4, 8};  // Thread configs
    double performance[4];              // For storing execution times
    long memory_usage[4];               // For storing memory usage

    struct timespec c_start, c_end;

    // Loop through the different thread counts
    for (int t = 0; t < 4; t++) {
        NUM_THREADS = thread_count[t];
        chunk_size = ARRAY_SIZE / NUM_THREADS;
        if (NUM_THREADS == 1){
            printf(" - %d THREAD:\n", NUM_THREADS);
        } else {
            printf(" - %d THREADS:\n", NUM_THREADS);
        }

        // Generate & Fill Array
        srand(42);
        for (int i = 0; i < ARRAY_SIZE; i++) {
            array[i] = rand() % 100;
        }

        // Print Array
        printf("    - Array (first 20 elements):\n\t");
        for (int i = 0; i < 20; i++) {
            printf("%d ", array[i]);
        }
        printf("\n\n");

        // Record memory and time before execution
        long mem_before = get_memory_usage();
        clock_gettime(CLOCK_MONOTONIC, &c_start);

        // ---- Map Phase ------------------------------------------------------
        // Creates threads to process chunks
        printf("    - Finding Global Max:\n");
        global_max = array[0];
        pthread_t threads[NUM_THREADS];
        for (int i = 0; i < NUM_THREADS; i++) {
            int *arg = malloc(sizeof(int));
            *arg = i;
            pthread_create(&threads[i], NULL, find_local_max, arg);
        }

        // Wait for all threads to complete
        for (int i = 0; i < NUM_THREADS; i++) {
            pthread_join(threads[i], NULL);
        }
        printf("\n\t - All threads finished -\n");

        // Output Resutl
        printf("\n    - Global Max: %d\n", global_max);

        // Record memory and time after execution
        clock_gettime(CLOCK_MONOTONIC, &c_end);
        long mem_after = get_memory_usage();

        // Calculate exeuction time
        double execution_time = (c_end.tv_sec - c_start.tv_sec) + (c_end.tv_nsec - c_start.tv_nsec) / 1e9;
        printf("\n    - Execution Time: %f sec", execution_time);
        performance[t] = execution_time;

        // Caluclate memory usage
        memory_usage[t] = mem_after - mem_before;
        printf("\n\n    - Memory Before: %ld KB\n", mem_before);
        printf("    - Memory After: %ld KB\n", mem_after);
        printf("    - Memory Delta: %ld KB", mem_after - mem_before);

        printf("\n------------------------------------------------------------------------------------------------------------------------\n");
    }
    pthread_mutex_destroy(&mutex); // Destory mutex for all threads

    // Print performance summary for all threads
    printf("\nPerformance Summary:\n");
    printf("Thread\t   Time (s)\tMem Delta (KB)\n");
    for (int t = 0; t < 4; t++) {
        printf("%d\t   %.6f\t%ld\n", thread_count[t], performance[t], memory_usage[t]);
    }

    return 0;
}

Overwriting max_value_multithreading.c


### Results

In [192]:
!gcc max_value_multithreading.c -o max_value_multithreading -lpthread
!./max_value_multithreading

------------------------------------------------------------------------------------------------------------------------
 - 1 THREAD:
    - Array (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Finding Global Max:
	Thread 0: Finding local max in 0 to 131071

	 - All threads finished -

    - Global Max: 99

    - Execution Time: 0.000953 sec

    - Memory Before: 1924 KB
    - Memory After: 2464 KB
    - Memory Delta: 540 KB
------------------------------------------------------------------------------------------------------------------------
 - 2 THREADS:
    - Array (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Finding Global Max:
	Thread 0: Finding local max in 0 to 65535
	Thread 1: Finding local max in 65536 to 131071

	 - All threads finished -

    - Global Max: 99

    - Execution Time: 0.000475 sec

    - Memory Before: 2464 KB
    - Memory After: 2476 KB
    - Memory Delta: 12 KB
-------------

## Multi-Processing

In [193]:
%%writefile max_value_multiprocessing.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <sys/mman.h>
#include <semaphore.h>
#include <time.h>

// Array Size
#define ARRAY_SIZE 131072

// Global Variables
int *array;
int chunk_size;
int *global_max;
sem_t *mutex;
long *child_mem;
int NUM_PROCESSES;

// Returns current memory usage by program
long get_memory_usage() {
    FILE* fp = fopen("/proc/self/status", "r");
    if (fp == NULL) {
        perror("Error opening /proc/self/status");
        return 0;
    }
    char line[256];
    long vm_rss = -1;
    while (fgets(line, sizeof(line), fp) != NULL) {
        if (strncmp(line, "VmRSS:", 6) == 0) {
            sscanf(line + 6, "%ld", &vm_rss);
            break;
        }
    }

    fclose(fp);
    return vm_rss;
}

// Computes maximum value within assigned chunk
void find_local_max(int id) {

    // Determine start and end for thread chunk
    int start = id * chunk_size;
    int end = 0;
    if (id == NUM_PROCESSES - 1) {
                end = ARRAY_SIZE - 1;
            } else {
                end = start + chunk_size - 1;
            }

    printf("\tProcess %d (PID=%d): sorting %d to %d\n", id, getpid(), start, end);
    fflush(stdout);

    // Compute local max
    int local_max = array[start];
    for (int i = start + 1; i <= end; i++) {
        if (array[i] > local_max) {
            local_max = array[i];
        }
    }

    // ---- Reduce Phase -------------------------------------------------------
    // Updates global max protected with mutex
    sem_wait(mutex);
    if (local_max > *global_max) {
        *global_max = local_max;
    }
    sem_post(mutex);

    // Collect child process memory usage
    child_mem[id] = get_memory_usage();

    _exit(0);
}

// Main Method
int main(void) {
    // Shared Memory
    array = mmap(NULL, ARRAY_SIZE * sizeof(int), PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);
    global_max = mmap(NULL, sizeof(int), PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);
    mutex = mmap(NULL, sizeof(sem_t), PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);
    child_mem = mmap(NULL, 8 * sizeof(long), PROT_READ | PROT_WRITE, MAP_SHARED | MAP_ANONYMOUS, -1, 0);

    printf("------------------------------------------------------------------------------------------------------------------------\n");
    int process_count[] = {1, 2, 4, 8}; // Thread configs
    double performance[4];              // For storing execution times
    long memory_usage[4];               // For storing memory Usage

    struct timespec c_start, c_end;

    // Loop through the different process counts
    for (int p = 0; p < 4; p++) {
        *global_max = array[0];
        NUM_PROCESSES = process_count[p];
        chunk_size = ARRAY_SIZE / NUM_PROCESSES;
        if (NUM_PROCESSES == 1){
            printf(" - %d PROCESS:\n", NUM_PROCESSES);
        } else {
            printf(" - %d PROCESSES:\n", NUM_PROCESSES);
        }

        // Generate & Fill Array
        srand(42);
        for (int i = 0; i < ARRAY_SIZE; i++) {
            array[i] = rand() % 100;
        }

        // Print Array
        printf("    - Array (first 20 elements):\n\t");
        for (int i = 0; i < 20; i++) {
            printf("%d ", array[i]);
        }
        printf("\n\n");

        // Record memory and time before execution
        long mem_before = get_memory_usage();
        clock_gettime(CLOCK_MONOTONIC, &c_start);

        // ---- Map Phase ------------------------------------------------------
        // Create processes to process chunks
        printf("    - Finding Global Max:\n");
        *global_max = array[0];
        sem_init(mutex, 1, 1);
        pid_t pids[NUM_PROCESSES];
        for (int i = 0; i < NUM_PROCESSES; i++) {
            pids[i] = fork();
            if(pids[i] == 0) {
                find_local_max(i);
                _exit(0);
            }
        }

        // Wait for all child process to finish
        for (int i = 0; i < NUM_PROCESSES; i++) {
            waitpid(pids[i], NULL, 0);
        }
        printf("\n\t - All processess finished -\n");

        // Output Result
        printf("\n    - Global Max: %d\n", *global_max);

        // Record memory and time after execution
        long mem_after = get_memory_usage();
        clock_gettime(CLOCK_MONOTONIC, &c_end);

        // Calculate total memory usage
        long total_child_mem = 0;
        for (int i = 0; i < NUM_PROCESSES; i++) {
            total_child_mem += child_mem[i];
        }
        memory_usage[p] = total_child_mem;
        printf("\n    - Total memory used by children: %ld KB\n", total_child_mem);

        // Calculate parent memory usage
        double execution_time = (c_end.tv_sec - c_start.tv_sec) + (c_end.tv_nsec - c_start.tv_nsec) / 1e9;
        printf("\n    - Execution Time: %f sec", execution_time);
        printf("\n\n    - Memory Before: %ld KB\n", mem_before);
        printf("    - Memory After: %ld KB\n", mem_after);
        printf("    - Memory Delta: %ld KB", mem_after - mem_before);

        printf("\n------------------------------------------------------------------------------------------------------------------------\n");
        performance[p] = execution_time;

    }

    // Print performance data for each process count
    printf("\nPerformance Summary:\n");
    printf("Processes  Time (s)\tMem Usage (KB)\n");
    for (int p = 0; p < 4; p++) {
        printf("%d\t   %.6f\t%ld\n", process_count[p], performance[p], memory_usage[p]);
    }

    // Destory semaphore and release memory
    sem_destroy(mutex);
    munmap(array, ARRAY_SIZE * sizeof(int));
    munmap(global_max, sizeof(int));
    munmap(mutex, sizeof(sem_t));
    munmap(child_mem, 8 * sizeof(long));
    return 0;
}

Overwriting max_value_multiprocessing.c


### Results

In [188]:
!gcc max_value_multiprocessing.c -o max_value_multiprocessing -lpthread
!./max_value_multiprocessing

------------------------------------------------------------------------------------------------------------------------
 - 1 PROCESS:
    - Array (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Finding Global Max:
	Process 0 (PID=140858): sorting 0 to 131071

	 - All processess finished -

    - Global Max: 99

    - Total memory used by children: 1324 KB

    - Execution Time: 0.000882 sec

    - Memory Before: 1972 KB
    - Memory After: 2100 KB
    - Memory Delta: 128 KB
------------------------------------------------------------------------------------------------------------------------
 - 2 PROCESSES:
    - Array (first 20 elements):
	66 40 81 41 12 58 21 40 35 43 74 43 17 4 96 62 92 48 98 59 

    - Finding Global Max:
	Process 0 (PID=140859): sorting 0 to 65535
	Process 1 (PID=140860): sorting 65536 to 131071

	 - All processess finished -

    - Global Max: 99

    - Total memory used by children: 2200 KB

    - Execution Time: 0.0007